In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import pydicom
import os
from tqdm import tqdm

In [2]:
import torchvision

In [3]:
import os
import cv2
import pdb
import time
import warnings
import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader, Dataset, sampler
from matplotlib import pyplot as plt
from albumentations import (HorizontalFlip, ShiftScaleRotate, Normalize, Resize, Compose, GaussNoise)
from albumentations.torch import ToTensor
!pip install git+https://github.com/qubvel/segmentation_models.pytorch > /dev/null 2>&1 # run bash cmd without output.
import segmentation_models_pytorch as smp
warnings.filterwarnings("ignore")

In [4]:
path1 = '../input/chexnet/repository/zoogzog-chexnet-bbe30b5/models/m-25012018-123527.pth.tar'
y = torch.load(path1)

In [5]:
class DenseNet121(nn.Module):

    def __init__(self, classCount, isTrained):
        super(DenseNet121, self).__init__()
        self.densenet121 = torchvision.models.densenet121(pretrained=isTrained)
        kernelCount = self.densenet121.classifier.in_features
        self.densenet121.classifier = nn.Sequential(nn.Linear(kernelCount, classCount), nn.Sigmoid())

    def forward(self, x):
        x = self.densenet121(x)
        return x

In [6]:
from collections import OrderedDict
e = [str(i) for i in range(10)]
new_dict = OrderedDict()
for k,v in y['state_dict'].items():
    b = k.split('.')
    if b[-2] in e:
        b[-3] = b[-3]+b[-2]
        del(b[-2])
        s = '.'.join(b)
        new_dict[s] = v
    else:
        new_dict[k] = v 
new_dict['module.densenet121.classifier.0.weight'] = new_dict.pop('module.densenet121.classifier0.weight',None) 
new_dict['module.densenet121.classifier.0.bias'] = new_dict.pop('module.densenet121.classifier0.bias',None)

In [7]:
model2 = DenseNet121(14, True).cuda()
model2 = torch.nn.DataParallel(model2).cuda()
modelCheckpoint = torch.load(path1)
model2.load_state_dict(new_dict)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /tmp/.cache/torch/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 84.9MB/s]


<All keys matched successfully>

In [8]:
model2 = model2.module.densenet121.features
model2.eval()

Sequential(
  (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU(inplace=True)
  (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (denseblock1): _DenseBlock(
    (denselayer1): _DenseLayer(
      (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace=True)
      (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (denselayer2): _DenseLayer(
      (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv1): Conv2d(96, 128, ke

In [9]:
# Show some images
from glob import glob
train_glob = '../input/penumo-test-train/dicom-images-train/dicom-images-train/*/*/*.dcm'
train_fns = sorted(glob(train_glob))
test_stg1_fns = sorted(glob('../input/penumo-test-train/dicom-images-test/dicom-images-test/*/*/*.dcm'))

print(len(train_fns))
print(len(test_stg1_fns))

10675
1372


In [10]:
#Create dataset
class SIIM_pneumo_dataset(Dataset):
    
    def __init__(self,df,size,phase,mean,std):
        self.df = df
        self.size = size 
        self.std = std
        self.mean = mean 
        self.transforms = get_transforms(phase,size,mean,std)
    
    def __getitem__(self,idx):
        path = self.df.iloc[idx,1]
        dset = pydicom.dcmread(path)
        img = dset.pixel_array
        img = np.expand_dims(img,axis = 2)
        b = img.copy()
        c = img.copy()
        img = np.concatenate((img,b,c),axis = 2)
        mask = np.zeros([1024,1024])
        rle = self.df.iloc[idx,2]
        if rle != '-1':
            mask += rle2mask(rle,1024,1024).T
        mask = (mask >= 1).astype('float32') 
        augmented = self.transforms(image = img, mask = mask)
        img = augmented['image']
        mask = augmented['mask']
        return(img,mask) 
   
    def __len__(self):
        return(len(self.df))
    
def get_transforms(phase,size,mean,std):
    list_transforms = []
    if phase == 'train':
        list_transforms.extend([ShiftScaleRotate(shift_limit=0, scale_limit=0.1,rotate_limit=10,p=0.5,border_mode=cv2.BORDER_CONSTANT)])
    list_transforms.extend([Normalize(mean,std,p = 1),Resize(size,size),ToTensor()])
    trfms = Compose(list_transforms)
    return(trfms)    

In [11]:
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
size = 1024
list_transforms = []
list_transforms.extend([Normalize(mean,std,p = 1),Resize(size,size),ToTensor()])
trfms = Compose(list_transforms)

In [12]:
path1 = train_fns[0]
dset = pydicom.dcmread(path1)
img = dset.pixel_array
img = np.expand_dims(img,axis = 2)
b = img.copy()
c = img.copy()
img = np.concatenate((img,b,c),axis = 2)
augmented = trfms(image = img)
img = augmented['image']
img = img.unsqueeze(0)

Prep the datasets

In [13]:
path_stg2 = '../input/siim-acr-pneumothorax-segmentation/stage_2_train.csv'
df_stg2 = pd.read_csv(path_stg2)
df_stg2.head()

,ImageId,EncodedPixels
0,1.2.276.0.7230010.3.1.4.8323329.6904.151787520...,-1
1,1.2.276.0.7230010.3.1.4.8323329.13666.15178752...,557374 2 1015 8 1009 14 1002 20 997 26 990 32 ...
2,1.2.276.0.7230010.3.1.4.8323329.11028.15178752...,-1
3,1.2.276.0.7230010.3.1.4.8323329.10366.15178752...,514175 10 1008 29 994 30 993 32 991 33 990 34 ...
4,1.2.276.0.7230010.3.1.4.8323329.10016.15178752...,592184 33 976 58 956 73 941 88 926 102 917 109...


In [14]:
d = {}
d['ImageId'] = []
d['path'] = []
for f in tqdm(train_fns):
    dcm_data = pydicom.dcmread(f)
    x = dcm_data.SOPInstanceUID
    d['ImageId'].append(x)
    d['path'].append(f)
metadata_df = pd.DataFrame(d)
metadata_df.columns = ['ImageId','path']

In [15]:
d = {}
d['ImageId'] = []
d['path'] = []
for f in tqdm(test_stg1_fns):
    dcm_data = pydicom.dcmread(f)
    x = dcm_data.SOPInstanceUID
    d['ImageId'].append(x)
    d['path'].append(f)
metadata_2 = pd.DataFrame(d)
metadata_2.head()

,ImageId,path
0,1.2.276.0.7230010.3.1.4.8323329.5797.151787519...,../input/penumo-test-train/dicom-images-test/d...
1,1.2.276.0.7230010.3.1.4.8323329.5798.151787519...,../input/penumo-test-train/dicom-images-test/d...
2,1.2.276.0.7230010.3.1.4.8323329.5799.151787519...,../input/penumo-test-train/dicom-images-test/d...
3,1.2.276.0.7230010.3.1.4.8323329.580.1517875163...,../input/penumo-test-train/dicom-images-test/d...
4,1.2.276.0.7230010.3.1.4.8323329.5800.151787519...,../input/penumo-test-train/dicom-images-test/d...


In [16]:
df3 = pd.concat([metadata_df,metadata_2])
df4 = pd.merge(df3,df_stg2,on = 'ImageId')
has_pneumo = []
for j in range(len(df4)):
    if df4.iloc[j,2] == '-1':
        has_pneumo.append(0)
    else:
        has_pneumo.append(1)
df4['has_pneumo'] = has_pneumo  
df4.head()

,ImageId,path,EncodedPixels,has_pneumo
0,1.2.276.0.7230010.3.1.4.8323329.1000.151787516...,../input/penumo-test-train/dicom-images-train/...,-1,0
1,1.2.276.0.7230010.3.1.4.8323329.10000.15178752...,../input/penumo-test-train/dicom-images-train/...,-1,0
2,1.2.276.0.7230010.3.1.4.8323329.10001.15178752...,../input/penumo-test-train/dicom-images-train/...,-1,0
3,1.2.276.0.7230010.3.1.4.8323329.10002.15178752...,../input/penumo-test-train/dicom-images-train/...,-1,0
4,1.2.276.0.7230010.3.1.4.8323329.10003.15178752...,../input/penumo-test-train/dicom-images-train/...,-1,0


Create the dataloader

In [17]:
#Create dataset
class SIIM_pneumo_dataset(Dataset):
    
    def __init__(self,df,size,phase,mean,std):
        self.df = df
        self.size = size 
        self.std = std
        self.mean = mean 
        self.transforms = get_transforms(phase,size,mean,std)
    
    def __getitem__(self,idx):
        path = self.df.iloc[idx,1]
        dset = pydicom.dcmread(path)
        img = dset.pixel_array
        img = np.expand_dims(img,axis = 2)
        b = img.copy()
        
        c = img.copy()
        img = np.concatenate((img,b,c),axis = 2)
        mask = np.zeros([1024,1024])
        rle = self.df.iloc[idx,2]
        if rle != '-1':
            mask += rle2mask(rle,1024,1024).T
        mask = (mask >= 1).astype('float32') 
        augmented = self.transforms(image = img, mask = mask)
        img = augmented['image']
        mask = augmented['mask']
        return(img,mask) 
   
    def __len__(self):
        return(len(self.df))
    
def get_transforms(phase,size,mean,std):
    list_transforms = []
    if phase == 'train':
        list_transforms.extend([ShiftScaleRotate(shift_limit=0, scale_limit=0.1,
                    rotate_limit=10, # rotate
                    p=0.5,
                    border_mode=cv2.BORDER_CONSTANT
                )])
    list_transforms.extend([Normalize(mean,std,p = 1),Resize(size,size),ToTensor()])
    trfms = Compose(list_transforms)
    return(trfms)    

def provider(fold,total_folds,df,phase,size,mean=None,std=None,batch_size=8):
    df_with_mask = df[df["has_pneumo"] != 0]
    df_no_mask = df[df["has_pneumo"] == 0]
    df_no_mask_sampled = df_no_mask.sample(len(df_with_mask))
    df = pd.concat([df_with_mask, df_no_mask_sampled])
    #NOTE: equal number of positive and negative cases are chosen.
    kfold = StratifiedKFold(total_folds, shuffle=True, random_state=69)
    train_idx, val_idx = list(kfold.split(
        df["ImageId"], df["has_pneumo"]))[fold]
    train_df, val_df = df.iloc[train_idx], df.iloc[val_idx]
    df = train_df if phase == "train" else val_df
    # NOTE: total_folds=5 -> train/val : 80%/20%
    image_dataset = SIIM_pneumo_dataset(df,size,phase,mean,std)
    dataloader = DataLoader(image_dataset,batch_size=batch_size,pin_memory=True,shuffle=True,)
    return dataloader

Training the model

In [18]:
# Utility functions
def rle2mask(rle, width, height):
    mask= np.zeros(width* height)
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        current_position += start
        mask[current_position:current_position+lengths[index]] = 255
        current_position += lengths[index]

    return mask.reshape(width, height)

In [19]:
def dice_loss(x,target):
    x = torch.sigmoid(x)
    x = x.view(-1)
    target = target.view(-1)
    intersection = 2*((x*target).sum()) + 1
    sum1 = (target + x).sum() + 1 
    loss = intersection/sum1
    return(loss)

class loss(nn.Module):
    
    def __init__(self):
        super().__init__()
        
    def forward(self,input,target):
        loss = -torch.log(dice_loss(input,target))
        return(loss)
        

In [20]:
# Training the model
net_ans = []
class Trainer(object):
    def __init__(self,model,model2,net_ans):
        self.fold = 1
        self.total_folds = 5
        self.df = metadata_df
        self.phases = ['train','val']
        self.size = 1024
        self.num_epochs = 1
        self.lr = 5e-4
        self.batch_size = {'train':1,'val':1}
        self.dataloaders = {phase : provider(fold = 1, total_folds = 5,df = df4, 
                                             phase = phase,size = 1024,mean= 0.485,std= 0.225, batch_size=self.batch_size[phase]) for phase in self.phases}
        self.accumulation_steps = 32 // self.batch_size['train']
        self.criterion = loss()
        self.ngpu = 1
        self.device = torch.device("cuda:0" if (torch.cuda.is_available() and self.ngpu > 0) else "cpu")
        torch.set_default_tensor_type("torch.cuda.FloatTensor")
        self.net = model
        self.best_loss = float("inf")
        self.optimizer = optim.Adam(self.net.parameters(), lr=self.lr)
        self.scheduler = ReduceLROnPlateau(self.optimizer, mode="min", patience=3, verbose=True)
        self.net = self.net.to(self.device)
        self.model2 = model2
        self.model2 = self.model2.to(self.device)
        self.net_ans = net_ans
        
    def forward(self,images,targets):
        images = images.to(self.device)
        masks = targets.to(self.device)
        output = self.net(images)
        dice = self.criterion(output,masks)
        return(output,dice)
    
    def iterate(self,phase):
        dataloader = self.dataloaders[phase]
        self.net.train(phase == 'train')
        running_loss = 0.0
        for itr,batch in enumerate(dataloader):
            images,target = batch
            images = images.to(self.device)
            out1 = self.model2(images)
            self.net_ans.append(out1)
#            output,loss = self.forward(out1,target)
#            loss = loss/self.accumulation_steps
#            loss.backward()
#            if (itr+1)%self.accumulation_steps == 0:
#                self.optimizer.step()
#                self.optimizer.zero_grad()
#            running_loss += loss.item()
#        epoch_loss = (running_loss*self.accumulation_steps)/len(dataloader)  
#        print("Loss: %0.4f" % (epoch_loss))
#        return(epoch_loss)
        
    def start(self):
        for epoch in range(self.num_epochs):
            print('Epoch {}/{}'.format(epoch,self.num_epochs - 1))
            self.iterate('train')
#            state = {'epoch':epoch,'best_loss':self.best_loss,'state_dict':self.net.state_dict(),'optimizer':self.optimizer.state_dict()}
#            val_loss = self.iterate('val')
#            self.scheduler.step(val_loss)
#            if val_loss < self.best_loss:
#                print("******** New optimal found, saving state ********")
#                state["best_loss"] = self.best_loss = val_loss
#                torch.save(state, "./model.pth")
            print()

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [22]:
#import torch.nn as nn
#import torch.nn.functional as F
#
#class Net(nn.Module):
#
#    def __init__(self):
#        super(Net, self).__init__()
#        # 32 * 32
#        self.convt1 = nn.ConvTranspose2d(1024,512,2,2,0)
#        self.batch1 = nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
#        self.act1 = nn.ReLU(inplace = True)
#        # 64 * 64
#        self.convt2 = nn.ConvTranspose2d(512,256,2,2,0)
#        self.batch2 = nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
#        self.act2 = nn.ReLU(inplace = True)
#        # 128 * 128
#        self.convt3 = nn.ConvTranspose2d(256,64,2,2,0)  
#        self.batch3 = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
#        self.act3 = nn.ReLU(inplace = True)
#        #256 * 256
#        self.convt4 = nn.ConvTranspose2d(64,16,2,2,0)  
#        self.batch4 = nn.BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
#        self.act4 = nn.ReLU(inplace = True)
#        #512 * 512
#        self.convt5 = nn.ConvTranspose2d(16,4,2,2,0)  
#        self.batch5 = nn.BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
#        self.act5 = nn.ReLU(inplace = True)
#        #1024 * 1024
#        self.convt6 = nn.ConvTranspose2d(4,1,1,1,0)  
#        #1024 * 1024
#            
#    def forward(self, x):
#        x = self.convt1(x)
#        x = self.batch1(x)
#        x = self.act1(x)
#        x = self.convt2(x)
#        x = self.batch2(x)
#        x = self.act2(x)
#        x = self.convt3(x)
#        x = self.batch3(x)
#        x = self.act3(x)
#        x = self.convt4(x)
#        x = self.batch4(x)
#        x = self.act4(x)
#        x = self.convt5(x)
#        x = self.batch5(x)
#        x = self.act5(x)
#        x = self.convt6(x)
#        return x
#
#    def num_flat_features(self, x):
#        size = x.size()[1:]  # all dimensions except the batch dimension
#        num_features = 1
#        for s in size:
#            num_features *= s
#        return num_features
#
#net = Net().cuda()
#print(net)

In [23]:
#NET2

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 32 * 32
        self.convt1 = nn.ConvTranspose2d(1024,64,4,4,0)
        self.batch1 = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.act1 = nn.ReLU(inplace = True)
        # 128 * 128
        self.convt2 = nn.ConvTranspose2d(64,4,8,8,0)
        self.batch2 = nn.BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.act2 = nn.ReLU(inplace = True)
        # 1024 * 1024
        self.convt3 = nn.ConvTranspose2d(4,1,1,1,0)  
        #1024 * 1024
        
            
    def forward(self, x):
        x = self.convt1(x)
        x = self.batch1(x)
        x = self.act1(x)
        x = self.convt2(x)
        x = self.batch2(x)
        x = self.act2(x)
        x = self.convt3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net().cuda()
print(net)

Net(
  (convt1): ConvTranspose2d(1024, 64, kernel_size=(4, 4), stride=(4, 4))
  (batch1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (convt2): ConvTranspose2d(64, 4, kernel_size=(8, 8), stride=(8, 8))
  (batch2): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act2): ReLU(inplace=True)
  (convt3): ConvTranspose2d(4, 1, kernel_size=(1, 1), stride=(1, 1))
)


In [24]:
trainer = Trainer(net,model2,net_ans)
trainer.start()

Epoch 0/0


RuntimeError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 15.90 GiB total capacity; 15.10 GiB already allocated; 9.88 MiB free; 117.05 MiB cached)

In [25]:
np.save('./net_ans.py',net_ans)